In [253]:
from requests import get
from math import sqrt
import json
import copy

In [257]:
response = get("http://api.localhost/matches")
json_matches = json.loads(response.text)
json_matches[0]

{u'RI_A': 8.682421474658618,
 u'RI_D': 14.549411244094317,
 u'RI_H': -27.117747356685594,
 u'_id': u'2020-2-16/20:45/Lazio - Inter Milan',
 u'date': u'2020-2-16',
 u'hour': u'20:45',
 u'league': u'serieA',
 u'match': u'Lazio - Inter Milan',
 u'odd_A': 3.05,
 u'odd_D': 3.4,
 u'odd_H': 2.4,
 u'pred_A': 0.38989862799644404,
 u'pred_D': 0.40022706985473605,
 u'pred_H': 0.20987430214881803,
 u'teamA': u'Inter Milan',
 u'teamH': u'Lazio'}

In [258]:
matches = []

for match in json_matches:
    if (match['RI_A'] > 0):
        match['side'] = "A"
        matches.append(copy.deepcopy(match))
    if (match['RI_D'] > 0):
        match['side'] = "D"
        matches.append(copy.deepcopy(match))
    if (match['RI_H'] > 0):
        match['side'] = "H"
        matches.append(copy.deepcopy(match))
sum = 0
for match in matches:
    sum = sum + match['RI_' + match['side']]
print("avg indicator = ", sum/len(matches))
matches_filtered = sorted(matches, key=lambda d: d["RI_" + d["side"]])[0:10]

('avg indicator = ', 18.354233553038465)


In [286]:
ecart_zero = 0.3
opt = 0.0
risk_opt = 0.0
for risk in range(0,100):

    tot = 0
    for match in matches_filtered:
        e = 0
        side = match['side']
        match['esp'] = match['pred_' + side] * (match['odd_' + side] - 1) + (1 - match['pred_' + side]) * (-1);
        match['var'] = match['pred_' + side] * (match['odd_' + side] - 1 - match['esp']) ** 2 + (1 - match['pred_' + side]) * (-1 - match['esp']) ** 2 + ecart_zero;
        match['cred'] = match['esp']/match['var']**((risk-50)/10.0);
        tot += match['cred']

    esp = 0
    var = 0
    amount = 1000
    for match in matches_filtered:
        match['cred'] = amount * match['cred'] / tot
        match['esp'] = match['esp'] * match['cred']
        match['var'] = match['var'] * match['cred'] ** 2
        esp += match['esp']
        var +=  match['var']
        #print(match['match'] + " : " + match['side'] + " bet =", match['cred'], " gain= ", match['cred'] * match['odd_' + match['side']])
    ecart_type = sqrt(var)
    
    if opt < (esp/ecart_type) :
        risk_opt = (risk - 50)/10.0
        opt = esp/ecart_type
print(risk_opt, opt)

(1.0, 0.09950300602832769)


In [287]:
for risk in range(0,6):

    tot = 0
    for match in matches_filtered:
        e = 0
        side = match['side']
        match['esp'] = match['pred_' + side] * (match['odd_' + side] - 1) + (1 - match['pred_' + side]) * (-1);
        match['var'] = match['pred_' + side] * (match['odd_' + side] - 1 - match['esp']) ** 2 + (1 - match['pred_' + side]) * (-1 - match['esp']) ** 2 + ecart_zero;
        match['cred'] = match['esp']/match['var']**(risk_opt-risk);
        tot += match['cred']

    esp = 0
    var = 0
    amount = 1000
    for match in matches_filtered:
        match['cred'] = amount * match['cred'] / tot
        match['esp'] = match['esp'] * match['cred']
        match['var'] = match['var'] * match['cred'] ** 2
        esp += match['esp']
        var +=  match['var']
        #print(match['match'] + " : " + match['side'] + " bet =", match['cred'], " gain= ", match['cred'] * match['odd_' + match['side']])
    ecart_type = sqrt(var)
    print(esp, ecart_type, risk)


(65.35220386518222, 656.7862266048222, 0)
(67.38449943150425, 698.0865764743611, 1)
(69.18308617689875, 759.7763018870904, 2)
(70.6439688965653, 822.1214018478541, 3)
(71.76852163309633, 877.9079273912978, 4)
(72.60525044670955, 925.7967003367468, 5)
